In [ ]:
def min_edits(a: str, b: str) -> int:
    n, m = len(a), len(b)
    v = [0 for _ in range(n + m + 1)]

    for d in range(0, n + m + 1):
        for k in range(-d, d+1, 2):
            if k == -d or (k != d and v[k+1] > v[k-1]):
                x = v[k+1] # move downward
            else:
                x = v[k-1] + 1 # move rightwrad

            y = x - k
            while x < n and y < m and a[x] == b[y]:
                x, y = x + 1, y + 1

            if x == n and y == m:
                return d

            v[k] = x

    raise ValueError()


print("min edit distance: ", min_edits("ABC", ""))
print("min edit distance: ", min_edits("ABC", "DEF"))
print("min edit distance: ", min_edits("ABC", "ABB"))
print("min edit distance: ", min_edits("ABC", "AB"))
print("min edit distance: ", min_edits("ABC", "BC"))
print("min edit distance: ", min_edits("ABC", "CBA"))
print("min edit distance: ", min_edits("ABC", "ABC"))
print("min edit distance: ", min_edits("ABCABBA", "CBABAC"))


min edit distance:  3
min edit distance:  6
min edit distance:  2
min edit distance:  1
min edit distance:  1
min edit distance:  4
min edit distance:  0
min edit distance:  5


In [195]:
import textwrap
from collections import Counter
from enum import Enum
from IPython.display import display, HTML

class Edit(Enum):
    equal = "equal"
    insert = "insert"
    delete = "delete"


def render_edits(a: str, b: str, edits: list[Edit]):
    styles = textwrap.dedent(f"""
        .{Edit.equal.name}  {{white-space: pre;}}
        .{Edit.insert.name} {{white-space: pre;color: green;}}
        .{Edit.delete.name} {{white-space: pre;color: red;}}
    """)
    
    a_chars, b_chars = iter(a), iter(b)
    elements = []
    expected_length = len(edits) + Counter(edits).get(Edit.equal)
    if len(a) + len(b) != expected_length:
        msg = f"expected {expected_length} chars, actual {len(a) + len(b)}"
        raise ValueError(msg)
    for edit in edits:
        if edit == Edit.equal:
            elements.append(f"<div class='{edit.name}'>\t{next(a_chars)}\t{next(b_chars)}</div>")
        elif edit == Edit.insert:
            elements.append(f"<div class='{edit.name}'>+\t\t{next(b_chars)}</div>")
        elif edit == Edit.delete:
            elements.append(f"<div class='{edit.name}'>-\t{next(a_chars)}</div>")
    display(HTML(f"""<style>{styles}</style>{''.join(elements)}"""))

render_edits("ABC", "ABB", [Edit.equal, Edit.equal, Edit.delete, Edit.insert])

In [200]:
def myer_diff(a, b):
    n, m = len(a), len(b)
    v = [0 for _ in range(n + m + 1)]
    trace = []
    
    def backtrace(k: int):
        x, y = n, m
        diff = []
        
        while len(trace) > 0:
            tr = trace.pop()
            if tr[k+1] > tr[k-1]:
                prev_x, prev_y = tr[k+1], tr[k+1] - (k + 1)
                k = k + 1
            else:
                prev_x, prev_y = tr[k-1], tr[k-1] - (k - 1)
                k = k - 1
            
            while x > prev_x and y > prev_y:
                x, y = x - 1, y - 1
                diff.append(Edit.equal)
            
            if x > prev_x:
                diff.append(Edit.delete)
            else:
                diff.append(Edit.insert)

            x, y = prev_x, prev_y
        
        diff.reverse()
        return diff

    for d in range(1, n + m + 1):
        trace.append(list(v))
        for k in range(-d, d+1, 2):
            if (k == -d or (k < d and v[k+1] > v[k-1])):
                x = v[k+1] # move downward
            else:
                x = v[k-1] + 1 # move rightwrad

            y = x - k
            while x < n and y < m and a[x] == b[y]:
                x, y = x + 1, y + 1

            if x == n and y == m:
                return backtrace(k)

            v[k] = x

    raise ValueError("path not found")


render_edits("ABCABBA", "CBABAC", myer_diff("ABCABBA", "CBABAC"))


In [206]:
from collections import namedtuple
from dataclasses import dataclass

Point = namedtuple("Point", ["x", "y"])

@dataclass
class Box:
    origin: Point
    width: int
    height: int

    @property
    def left(self) -> int:
        return self.origin.x
    
    @property
    def right(self) -> int:
        return self.origin.x + self.width
    
    @property
    def top(self) -> int:
        return self.origin.y
    
    @property
    def bottom(self) -> int:
        return self.origin.y + self.height

def linear_myer(a: str, b: str):        

    def forward(box: Box, vf: list[int], vb: list[int], d: int):
        delta = box.width - box.height
        for k in reversed(range(-d, d+1, 2)):
            if (k == -d or (k != d and vf[k+1] > vf[k-1])):
                px = x = vf[k+1] # move downward
            else:
                px = vf[k-1] 
                x = px + 1 # move rightwrad

            y = box.top + ((x - box.left) - k)
            py = y - 1 if d != 0 and x == px else y

            while x < box.right and y < box.bottom and a[x] == b[y]:
                x, y = x + 1, y + 1

            vf[k] = x
            
            c = k - delta
            if delta % 2 == 1 and -d < c < d and y >= vb[c]:
                return Point(px, py), Point(x, y)

        return None

    def backward(box, vf, vb, d):
        delta = box.width - box.height
        for c in reversed(range(-d, d+1, 2)):
            if (c == -d or (c != d and vb[c-1] > vb[c+1])):
                py, px = vb[c+1], vb[c+1] - box.bottom + box.right + c + 1
            else:
                py, px = vb[c-1], vb[c-1] - box.bottom + box.right + c - 1

            x, y = px, py
            while x > box.left and y > box.top and a[x-1] == b[y-1]:
                x, y = x - 1, y - 1

            if py == vb[c+1]:
                x = x - 1 # move leftward
            else:
                y = y - 1 # move upward

            vb[c] = y
            k = c + delta
            if delta % 2 == 0 and -d <= k <= d and x <= vf[k]:
                return Point(x, y), Point(px, py)
        return None

    def find_snake(box: Box) -> tuple[Point, Point]:
        if box.width + box.height == 0:
            return None
        
        mrounds = (box.width + box.height + 1) // 2 + 1
        vf = [0 for _ in range(mrounds)]
        vb = [0 for _ in range(mrounds)]

        vf[1], vb[0] = box.left, box.bottom

        snake = forward(box, vf, vb, 0)
        if snake:
            return snake

        for d in range(1, mrounds):
            snake = forward(box, vf, vb, d) or backward(box, vf, vb,  d)
            if snake:
                return snake

        raise ValueError("snake not found")

    def find_path(box: Box):
        snake = find_snake(box)

        if not snake:
            return None

        head, tail = snake
        p0 = find_path(Box(box.origin, head.x - box.left, head.y - box.top)) or (head, )
        p1 = find_path(Box(tail, box.right - tail.x, box.bottom - tail.y)) or (tail, )

        return p0 + p1

    def walk_snake(path: tuple[Point]):
        diff = []
        cx, cy = path[0]
        for nx, ny in path[1:]:
            while nx > cx and ny > cy and a[cx] == b[cy]:
                cx, cy = cx + 1, cy + 1
                diff.append(Edit.equal)

            if nx - cx > ny - cy:
                cx += 1
                diff.append(Edit.delete)
            else:
                cy += 1
                diff.append(Edit.insert)

            while cx < nx and cy < ny and a[cx] == b[cy]:
                diff.append(Edit.equal)
                cx, cy = cx + 1, cy + 1

        return diff

    
    n, m = len(a), len(b)
    return walk_snake(find_path(Box(Point(0, 0), n, m)))


render_edits("ABCABBA", "CBABAC", linear_myer("ABCABBA", "CBABAC"))
